# Classifier
This model is meant to classify between Safaitic and other inscription types

First we import the necessary dependencies

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np


Then we create the training and the validation data generators

In [6]:
# create the trainng data generator
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    'data/allScripts/train',
    target_size=(150, 150),
    class_mode='binary'
)

# create validation data generator
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'data/allScripts/validation',
    target_size=(150, 150),
    class_mode='binary'
)

Found 590 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


Then we want to create the neural network and model. For this model, we will extend layers of an existing model (InceptionV3)

In [8]:
# Download the pre trained model
pre_trained_model = InceptionV3(input_shape=(150,150,3),weights='imagenet', include_top=False)

# Go through each layer in the pre-trained model and set as non-trainable (so that when we train our model, we don't update the existing weights)
for layer in pre_trained_model.layers:
    layer.trainable = False

# Find the last layer you'd like to use, and grab the output
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

# Add your custom layers under the chosen lasy layer
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(pre_trained_model.input, x)

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

We can then train the model:

In [9]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

C:\Users\moham\Desktop\Programming\machineLearning\inscriptions-ai\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step - accuracy: 0.6053 - loss: 8.7002

C:\Users\moham\Desktop\Programming\machineLearning\inscriptions-ai\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 531ms/step - accuracy: 0.6847 - loss: 4.3031 - val_accuracy: 0.8182 - val_loss: 0.4630
Epoch 2/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 479ms/step - accuracy: 0.7881 - loss: 0.5840 - val_accuracy: 0.8409 - val_loss: 0.3292
Epoch 3/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 441ms/step - accuracy: 0.8169 - loss: 0.4694 - val_accuracy: 0.5682 - val_loss: 1.1187
Epoch 4/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 445ms/step - accuracy: 0.8508 - loss: 0.3891 - val_accuracy: 0.9091 - val_loss: 0.2086
Epoch 5/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 445ms/step - accuracy: 0.9034 - loss: 0.2894 - val_accuracy: 0.8182 - val_loss: 0.3651
Epoch 6/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 435ms/step - accuracy: 0.8780 - loss: 0.2894 - val_accuracy: 0.9091 - val_loss: 0.2764
Epoch 7/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 441ms/step - accuracy: 0.8966 - loss: 0.2338 - val_accuracy: 0.7500 - val_loss: 0.5786
Epoch 8/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 440ms/step - accuracy: 0.8746 - loss: 0.2849 - val_accuracy: 0.7955 - val

We can now test the model

In [18]:
# Load and resize to the same size used in training (150x150 here)
img = image.load_img("classifier/test.jpg", target_size=(150, 150))

# Convert to array
img_array = image.img_to_array(img)

# Rescale (because you used rescale=1./255 in your generators)
# img_array = img_array / 255.0

# Add batch dimension (model expects shape: (batch_size, height, width, channels))
img_array = np.expand_dims(img_array, axis=0)


image_tensor = np.vstack([img_array])
classes = model.predict(image_tensor)
print(train_generator.class_indices)
print(classes)
print(classes[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
{'other': 0, 'safaitic': 1}
[[0.]]
[0.]
